In [ ]:
# Install required libraries
!pip install nibabel tensorflow h5py scikit-learn
!pip install keras
!pip install e3nn torch h5py numpy matplotlib
!pip install --upgrade e3nn
!pip install torch>=1.12.0
!pip install torchvision>=0.13.0
!pip install se3-transformer-pytorch
!pip install tqdm
!pip install h5py
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.2/447.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from se3_transformer_pytorch import SE3Transformer
from tqdm import tqdm
import h5py
import numpy as np
from torch.amp import GradScaler, autocast

/usr/local/lib/python3.11/dist-packages/se3_transformer_pytorch/irr_repr.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Jd = torch.load(str(path))


In [ ]:
# SE3 Classification Model
# SE3 Classification Model with Reduced Parameters
class SE3ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(SE3ClassificationModel, self).__init__()
        self.feature_proj = nn.Linear(1, 8)  # Smaller feature projection
        self.se3_transformer = SE3Transformer(
            dim=8,       # Minimal feature dimension
            depth=1,     # Single transformer layer
            heads=1,     # Single attention head
            num_degrees=1 # Minimal degrees of equivariance
        )
        self.fc = nn.Sequential(
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, features, coords):
        features = features.view(-1, 1)
        features = self.feature_proj(features)
        features = features.view(coords.shape[0], coords.shape[1], -1)
        transformed = torch.utils.checkpoint.checkpoint(self.se3_transformer, features, coords)
        pooled = torch.max(transformed, dim=1)[0]
        return self.fc(pooled)

In [ ]:
def load_data_from_h5(h5_file):
    with h5py.File(h5_file, "r") as f:
        coords_list, features_list, labels_list = [], [], []
        for label_name in f.keys():
            coords = torch.tensor(f[label_name]["coords"][:], dtype=torch.float32)
            features = torch.tensor(f[label_name]["features"][:], dtype=torch.float32)
            labels = torch.tensor(f[label_name]["labels"][:], dtype=torch.long)
            coords_list.append(coords)
            features_list.append(features)
            labels_list.append(labels)
        return coords_list, features_list, labels_list

def pad_point_clouds(coords_list, features_list, labels_list, max_points):
    """
    Pad point clouds, features, and labels to ensure uniform size across the dataset.

    Args:
        coords_list: List of coordinates arrays.
        features_list: List of features arrays.
        labels_list: List of labels (one label per sample).
        max_points: Maximum number of points to pad/truncate to.

    Returns:
        Padded coordinates, features, and labels as tensors.
    """
    padded_coords, padded_features, padded_labels = [], [], []

    for coords, features, label in zip(coords_list, features_list, labels_list):
        num_points = coords.shape[0]

        if num_points > max_points:
            # Truncate if the number of points exceeds max_points
            coords = coords[:max_points]
            features = features[:max_points]
        else:
            # Pad if the number of points is less than max_points
            padding = max_points - num_points
            coords = F.pad(coords, (0, 0, 0, padding), "constant", 0)
            features = F.pad(features, (0, 0, 0, padding), "constant", 0)

        # Ensure the label is a single integer
        if isinstance(label, torch.Tensor):
            label = label.item() if label.numel() == 1 else label[0].item()
        elif isinstance(label, np.ndarray):
            label = int(label[0]) if label.size == 1 else int(label.flat[0])
        else:
            label = int(label)

        # Add to the padded lists
        padded_coords.append(coords)
        padded_features.append(features)
        padded_labels.append(label)

    return (
        torch.stack(padded_coords),  # Stack coords into a single tensor
        torch.stack(padded_features),  # Stack features into a single tensor
        torch.tensor(padded_labels, dtype=torch.long),  # Convert labels to tensor
    )

In [ ]:
if __name__ == "__main__":
    # Load data from the HDF5 file
    h5_file = "dataset_150i_2048.h5"
    coords_list, features_list, labels_list = load_data_from_h5(h5_file)

    # Limit maximum points
    max_points = 2048  # Reduced further to save memory
    print(f"Using max_points: {max_points}")

    # Pad the data
    coords, features, labels = pad_point_clouds(coords_list, features_list, labels_list, max_points)

    # Split into training and validation sets
    split_idx = int(0.8 * len(labels))  # 80% for training
    train_coords, val_coords = coords[:split_idx], coords[split_idx:]
    train_features, val_features = features[:split_idx], features[split_idx:]
    train_labels, val_labels = labels[:split_idx], labels[split_idx:]

    # Create DataLoaders for training and testing
    # batch_size = 1  # Set batch size to 1 for minimal memory usage
    train_dataset = torch.utils.data.TensorDataset(train_coords, train_features, train_labels)
    val_dataset = torch.utils.data.TensorDataset(val_coords, val_features, val_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

    # Initialize the model, optimizer, and loss function
    num_classes = 3
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SE3ClassificationModel(num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # Function to check memory usage
    def check_memory(stage=""):
        allocated = torch.cuda.memory_allocated() / 1024**3
        reserved = torch.cuda.memory_reserved() / 1024**3
        print(f"[{stage}] Allocated: {allocated:.2f} GB, Reserved: {reserved:.2f} GB")

    # Training loop
    num_epochs = 5
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        correct_train = 0
        total_train = 0

        for coords, features, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            coords, features, labels = coords.to(device), features.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(features, coords)
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Calculate training accuracy
        train_accuracy = 100 * correct_train / total_train

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for val_coords, val_features, val_labels in val_loader:
                val_coords, val_features, val_labels = val_coords.to(device), val_features.to(device), val_labels.to(device)
                val_outputs = model(val_features, val_coords)
                loss = criterion(val_outputs, val_labels)

                val_loss += loss.item()
                _, predicted = torch.max(val_outputs.data, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted == val_labels).sum().item()

        # Calculate validation accuracy
        val_accuracy = 100 * correct_val / total_val

        # Print epoch results
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Training Loss: {epoch_loss/len(train_loader):.4f}, "
              f"Training Accuracy: {train_accuracy:.2f}%, "
              f"Validation Loss: {val_loss/len(val_loader):.4f}, "
              f"Validation Accuracy: {val_accuracy:.2f}%")

    # Save the trained model
    torch.save(model.state_dict(), "se3_classification_model_e5.pth")
    print("Model saved as se3_classification_model.pth")


Using max_points: 2048


Training Epoch 1/5:   0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


compute 0.pkl.gz... save 0.pkl.gz... done


Training Epoch 1/5: 100%|██████████| 2/2 [11:12<00:00, 336.13s/it]
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 1/5, Training Loss: 78.2115, Training Accuracy: 50.00%, Validation Loss: 54.4775, Validation Accuracy: 0.00%


Training Epoch 2/5:   0%|          | 0/2 [00:35<?, ?it/s]


KeyboardInterrupt: 